In [243]:
import pandas as pd
from transformers import pipeline
from bs4 import BeautifulSoup
import nltk
from nltk.stem.porter import *
from nltk.tokenize import word_tokenize
from nltk.tokenize.treebank import TreebankWordDetokenizer
nltk.download('punkt')

[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\ThibJacq\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!


True

In [244]:
nltk.download('stopwords', quiet=True)

True

In [245]:
df = pd.read_csv('./data/esa_csvs_raw.txt', sep='\t')

In [246]:
df.iloc[0]['csvText']

'[["Survey Date", "Aboriginal Communities", "Detail"], ["July 9 to 12, 2011", "Blueberry River First Nation <s>•</s>Halfway River First Nation <s>•</s>McLeod Lake Indian Band <s>•</s>North East Métis Association <s>•</s>", "Summer 2011 open water aquatic <s>•</s>assessments."], ["September 18 to October 10, 2011", "Blueberry River First Nation <s>•</s>Halfway River First Nation <s>•</s>McLeod Lake Indian Band <s>•</s>North East Métis Association <s>•</s>", "Fall 2011 open water aquatic assessments. <s>•</s>"], ["February 26 to March 12, 2012", "Blueberry River First Nation <s>•</s>Doig River First Nation <s>•</s>Halfway River First Nation <s>•</s>Kelly Lake Cree First Nation <s>•</s>McLeod Lake Indian Band <s>•</s>North East Métis Association <s>•</s>", "Winter 2012 aquatic habitat investigations. <s>•</s>"]]'

In [247]:
df_index = pd.read_csv('./data/ESA_website_ENG_2021_03_04_final.csv')

In [248]:
df_index.rename(columns={"Old CSV Filename": "csvFileName"}, inplace=True)

In [249]:
df_index.head()

,Index,Title,Content Type,Application Name,Application Short Name,Application Filing Date,Company Name,Commodity,File Name,ESA Folder URL,...,CSV Download URL,PDF Page Number,PDF Page Count,PDF Size,PDF Outline,Download folder name,Zipped Project Link,Missing CSV,CSV Filename,csvFileName
0,9134,TABLE 3 SUMMARY OF AQUATICS FIELD WORK AND ABO...,Table,Application for North Montney Project,North Montney,11/8/2013,NOVA Gas Transmission Ltd.,Gas,B2-16 ESA_Appendix_G_Part1of4 (A3Q6H2),https://apps.cer-rec.gc.ca/REGDOCS/Item/LoadRe...,...,http://www.cer-rec.gc.ca/esa-ees/nrthmntn/nrth...,14,48.0,5.87,No,nrthmntn,http://www.cer-rec.gc.ca/esa-ees/nrthmntn.zip,False,nrthmntn_table-3-summary-of-aquatics-field-wor...,1059614_14_lattice-v_1.csv
1,9135,TABLE 4 SUMMARY OF WATERCOURSE CROSSINGS ALONG...,Table,Application for North Montney Project,North Montney,11/8/2013,NOVA Gas Transmission Ltd.,Gas,B2-16 ESA_Appendix_G_Part1of4 (A3Q6H2),https://apps.cer-rec.gc.ca/REGDOCS/Item/LoadRe...,...,http://www.cer-rec.gc.ca/esa-ees/nrthmntn/nrth...,17,48.0,5.87,No,nrthmntn,http://www.cer-rec.gc.ca/esa-ees/nrthmntn.zip,False,nrthmntn_table-4-summary-of-watercourse-crossi...,1059614_17_lattice-v_1.csv
2,9136,TABLE 4 SUMMARY OF WATERCOURSE CROSSINGS ALONG...,Table,Application for North Montney Project,North Montney,11/8/2013,NOVA Gas Transmission Ltd.,Gas,B2-16 ESA_Appendix_G_Part1of4 (A3Q6H2),https://apps.cer-rec.gc.ca/REGDOCS/Item/LoadRe...,...,http://www.cer-rec.gc.ca/esa-ees/nrthmntn/nrth...,18,48.0,5.87,No,nrthmntn,http://www.cer-rec.gc.ca/esa-ees/nrthmntn.zip,False,nrthmntn_table-4-summary-of-watercourse-crossi...,1059614_18_lattice-v_1.csv
3,9137,TABLE 4 SUMMARY OF WATERCOURSE CROSSINGS ALONG...,Table,Application for North Montney Project,North Montney,11/8/2013,NOVA Gas Transmission Ltd.,Gas,B2-16 ESA_Appendix_G_Part1of4 (A3Q6H2),https://apps.cer-rec.gc.ca/REGDOCS/Item/LoadRe...,...,http://www.cer-rec.gc.ca/esa-ees/nrthmntn/nrth...,19,48.0,5.87,No,nrthmntn,http://www.cer-rec.gc.ca/esa-ees/nrthmntn.zip,False,nrthmntn_table-4-summary-of-watercourse-crossi...,1059614_19_lattice-v_1.csv
4,9138,TABLE 4 SUMMARY OF WATERCOURSE CROSSINGS ALONG...,Table,Application for North Montney Project,North Montney,11/8/2013,NOVA Gas Transmission Ltd.,Gas,B2-16 ESA_Appendix_G_Part1of4 (A3Q6H2),https://apps.cer-rec.gc.ca/REGDOCS/Item/LoadRe...,...,http://www.cer-rec.gc.ca/esa-ees/nrthmntn/nrth...,20,48.0,5.87,No,nrthmntn,http://www.cer-rec.gc.ca/esa-ees/nrthmntn.zip,False,nrthmntn_table-4-summary-of-watercourse-crossi...,1059614_20_lattice-v_1.csv


In [250]:
df_merged = df[df['csvFileName'].isin(list(df_index['csvFileName']))].reset_index(drop=True)

In [251]:
df_joined = pd.merge(df_index, df_merged, how='left', on='csvFileName')

In [252]:
df_joined.head()

,Index,Title,Content Type,Application Name,Application Short Name,Application Filing Date,Company Name,Commodity,File Name,ESA Folder URL,...,PDF Page Number,PDF Page Count,PDF Size,PDF Outline,Download folder name,Zipped Project Link,Missing CSV,CSV Filename,csvFileName,csvText
0,9134,TABLE 3 SUMMARY OF AQUATICS FIELD WORK AND ABO...,Table,Application for North Montney Project,North Montney,11/8/2013,NOVA Gas Transmission Ltd.,Gas,B2-16 ESA_Appendix_G_Part1of4 (A3Q6H2),https://apps.cer-rec.gc.ca/REGDOCS/Item/LoadRe...,...,14,48.0,5.87,No,nrthmntn,http://www.cer-rec.gc.ca/esa-ees/nrthmntn.zip,False,nrthmntn_table-3-summary-of-aquatics-field-wor...,1059614_14_lattice-v_1.csv,"[[""Survey Date"", ""Aboriginal Communities"", ""De..."
1,9135,TABLE 4 SUMMARY OF WATERCOURSE CROSSINGS ALONG...,Table,Application for North Montney Project,North Montney,11/8/2013,NOVA Gas Transmission Ltd.,Gas,B2-16 ESA_Appendix_G_Part1of4 (A3Q6H2),https://apps.cer-rec.gc.ca/REGDOCS/Item/LoadRe...,...,17,48.0,5.87,No,nrthmntn,http://www.cer-rec.gc.ca/esa-ees/nrthmntn.zip,False,nrthmntn_table-4-summary-of-watercourse-crossi...,1059614_17_lattice-v_1.csv,"[[""Field Site No."", ""Name, Legal Location"", ..."
2,9136,TABLE 4 SUMMARY OF WATERCOURSE CROSSINGS ALONG...,Table,Application for North Montney Project,North Montney,11/8/2013,NOVA Gas Transmission Ltd.,Gas,B2-16 ESA_Appendix_G_Part1of4 (A3Q6H2),https://apps.cer-rec.gc.ca/REGDOCS/Item/LoadRe...,...,18,48.0,5.87,No,nrthmntn,http://www.cer-rec.gc.ca/esa-ees/nrthmntn.zip,False,nrthmntn_table-4-summary-of-watercourse-crossi...,1059614_18_lattice-v_1.csv,"[[""Field Site No."", ""Name, Legal Location"", ..."
3,9137,TABLE 4 SUMMARY OF WATERCOURSE CROSSINGS ALONG...,Table,Application for North Montney Project,North Montney,11/8/2013,NOVA Gas Transmission Ltd.,Gas,B2-16 ESA_Appendix_G_Part1of4 (A3Q6H2),https://apps.cer-rec.gc.ca/REGDOCS/Item/LoadRe...,...,19,48.0,5.87,No,nrthmntn,http://www.cer-rec.gc.ca/esa-ees/nrthmntn.zip,False,nrthmntn_table-4-summary-of-watercourse-crossi...,1059614_19_lattice-v_1.csv,"[[""Field Site No."", ""Name, Legal Location"", ..."
4,9138,TABLE 4 SUMMARY OF WATERCOURSE CROSSINGS ALONG...,Table,Application for North Montney Project,North Montney,11/8/2013,NOVA Gas Transmission Ltd.,Gas,B2-16 ESA_Appendix_G_Part1of4 (A3Q6H2),https://apps.cer-rec.gc.ca/REGDOCS/Item/LoadRe...,...,20,48.0,5.87,No,nrthmntn,http://www.cer-rec.gc.ca/esa-ees/nrthmntn.zip,False,nrthmntn_table-4-summary-of-watercourse-crossi...,1059614_20_lattice-v_1.csv,"[[""Field Site No."", ""Name, Legal Location"", ..."


In [253]:
len(df_joined)

28891

In [254]:
del df
del df_index
del df_merged

In [255]:
df_joined.rename(columns={"csvText": "text"}, inplace=True)
df_joined['label'] = -1

In [256]:
df_joined["text"] = df_joined["Title"] + " " + df_joined["text"]
df_joined.drop(columns=["Title"], inplace=True)

In [257]:
# regex testing
df={'col1': ['cheese    cid26cid19cid21cid3 wow']}
cid_string = pd.DataFrame(data=d)

cid_string = cid_string.replace('cid\d+', '', regex=True).replace(' +', ' ', regex=True)
cid_string['col1'][0]

'cheese wow'

In [258]:
df_joined['text'] = df_joined['text'].apply(lambda x: BeautifulSoup(x, "html.parser").get_text(separator=' ')) # remove html tags
df_joined['text'] = df_joined['text'].replace('[^a-zA-Z0-9 ]', ' ', regex=True) # remove all non-alpha-numeric characters
df_joined['text'] = df_joined['text'].replace('\w{25,}', ' ', regex=True)
df_joined['text'] = df_joined['text'].replace('cid\d+', ' ', regex=True)
df_joined['text'] = df_joined['text'].replace(' s ', ' ', regex=True)
df_joined['text'] = df_joined['text'].replace(' +', ' ', regex=True) # remove all extra spaces in text

In [267]:
df_joined.text[103]

'Table 428 Assessed Value of an Average SingleFamily Home 2011 to 2013 Community 2011 2012 2013 2011 to 2013 increase Dawson Creek 212870 229000 238000 12 Fort St John 274035 305000 326000 19 Chetwynd 156170 218000 229000 47 Hudsons Hope 132270 161000 175000 32 Taylor 208098 247000 281000 35 SOURCE City of Dawson Creek 2013 '

In [268]:
from sklearn.model_selection import train_test_split
train_df, df_for_labeling = train_test_split(df_joined, test_size=1000, random_state=42, shuffle=True)

In [269]:
print(len(train_df))
print(len(df_for_labeling))

27891
1000


In [270]:
train_df.head(2)

,Index,Content Type,Application Name,Application Short Name,Application Filing Date,Company Name,Commodity,File Name,ESA Folder URL,Document Number,...,PDF Page Count,PDF Size,PDF Outline,Download folder name,Zipped Project Link,Missing CSV,CSV Filename,csvFileName,text,label
25896,35030,Table,Application for the Enbridge Northern Gateway ...,Northern Gateway,5/27/2010,"Northern Gateway Pipelines Inc., as General Pa...",Oil,B3-1 - Vol 6A P1 - Pipelines and Tank Terminal...,https://apps.cer-rec.gc.ca/REGDOCS/Item/LoadRe...,A1T0F1,...,184.0,4.85,Yes,nrthrngtwy,http://www.cer-rec.gc.ca/esa-ees/nrthrngtwy.zip,False,nrthrngtwy_table-2-6-estimated-barge-traffic-f...,620130_44_lattice-v_1.csv,Table 26 Estimated Barge Traffic for Marine Te...,-1
8409,17543,Table,Application for the 2017 NGTL System Expansion,2017 NGTL System Expansion,3/31/2015,NOVA Gas Transmission Ltd.,Gas,B3-33_ESA_Appendix_4_AQ_TDR_Part1of2 - A4K2T9,https://apps.cer-rec.gc.ca/REGDOCS/Item/LoadRe...,A4K2T9,...,39.0,5.67,Yes,2017ngtl,http://www.cer-rec.gc.ca/esa-ees/2017ngtl.zip,False,2017ngtl_table-14-alces-river-unit-addition-ai...,2758416_26_lattice-v_1.csv,TABLE 14 ALCES RIVER UNIT ADDITION AIR DISPERS...,-1


In [271]:
df_for_labeling.head(2)

,Index,Content Type,Application Name,Application Short Name,Application Filing Date,Company Name,Commodity,File Name,ESA Folder URL,Document Number,...,PDF Page Count,PDF Size,PDF Outline,Download folder name,Zipped Project Link,Missing CSV,CSV Filename,csvFileName,text,label
8889,18023,Table,Application for the 2017 NGTL System Expansion,2017 NGTL System Expansion,3/31/2015,NOVA Gas Transmission Ltd.,Gas,B2-15_ESA_Main_Sec_8_Part1of2 - A4K2S0,https://apps.cer-rec.gc.ca/REGDOCS/Item/LoadRe...,A4K2S0,...,472.0,6.50,Yes,2017ngtl,http://www.cer-rec.gc.ca/esa-ees/2017ngtl.zip,False,2017ngtl_table-8a-2-reasonably-forseeable-deve...,2758825_359_lattice-v_1.csv,TABLE 8A2 REASONABLY FORSEEABLE DEVELOPMENTS I...,-1
27753,36887,Table,Application for the Northwest Mainline Expansion,Northwest Mainline,4/29/2011,NOVA Gas Transmission Ltd.,Gas,B3-38 - 7212_App8_EPP - A1Y9J1,https://apps.cer-rec.gc.ca/REGDOCS/Item/LoadRe...,A1Y9J1,...,194.0,4.43,Yes,nrthwstmnln,http://www.cer-rec.gc.ca/esa-ees/nrthwstmnln.zip,False,nrthwstmnln_table-1-resource-specific-mitigati...,684515_22_lattice-v_1.csv,TABLE 1 RESOURCESPECIFIC MITIGATION TABLE Loca...,-1


In [272]:
df_for_labeling.to_csv('./mydata/esa_index_for_labeling.csv')

In [273]:
df_joined.to_csv('./data/esa_index_with_table_text_no_labels.csv')

In [274]:
df_for_labeling = pd.read_csv('./mydata/esa_index_for_labeling.csv', index_col=0)
df_train = pd.read_csv('./data/esa_index_with_table_text_no_labels.csv', index_col=0)

In [275]:
df_train.head()

,Index,Content Type,Application Name,Application Short Name,Application Filing Date,Company Name,Commodity,File Name,ESA Folder URL,Document Number,...,PDF Page Count,PDF Size,PDF Outline,Download folder name,Zipped Project Link,Missing CSV,CSV Filename,csvFileName,text,label
0,9134,Table,Application for North Montney Project,North Montney,11/8/2013,NOVA Gas Transmission Ltd.,Gas,B2-16 ESA_Appendix_G_Part1of4 (A3Q6H2),https://apps.cer-rec.gc.ca/REGDOCS/Item/LoadRe...,A3Q6H2,...,48.0,5.87,No,nrthmntn,http://www.cer-rec.gc.ca/esa-ees/nrthmntn.zip,False,nrthmntn_table-3-summary-of-aquatics-field-wor...,1059614_14_lattice-v_1.csv,TABLE 3 SUMMARY OF AQUATICS FIELD WORK AND ABO...,-1
1,9135,Table,Application for North Montney Project,North Montney,11/8/2013,NOVA Gas Transmission Ltd.,Gas,B2-16 ESA_Appendix_G_Part1of4 (A3Q6H2),https://apps.cer-rec.gc.ca/REGDOCS/Item/LoadRe...,A3Q6H2,...,48.0,5.87,No,nrthmntn,http://www.cer-rec.gc.ca/esa-ees/nrthmntn.zip,False,nrthmntn_table-4-summary-of-watercourse-crossi...,1059614_17_lattice-v_1.csv,TABLE 4 SUMMARY OF WATERCOURSE CROSSINGS ALONG...,-1
2,9136,Table,Application for North Montney Project,North Montney,11/8/2013,NOVA Gas Transmission Ltd.,Gas,B2-16 ESA_Appendix_G_Part1of4 (A3Q6H2),https://apps.cer-rec.gc.ca/REGDOCS/Item/LoadRe...,A3Q6H2,...,48.0,5.87,No,nrthmntn,http://www.cer-rec.gc.ca/esa-ees/nrthmntn.zip,False,nrthmntn_table-4-summary-of-watercourse-crossi...,1059614_18_lattice-v_1.csv,TABLE 4 SUMMARY OF WATERCOURSE CROSSINGS ALONG...,-1
3,9137,Table,Application for North Montney Project,North Montney,11/8/2013,NOVA Gas Transmission Ltd.,Gas,B2-16 ESA_Appendix_G_Part1of4 (A3Q6H2),https://apps.cer-rec.gc.ca/REGDOCS/Item/LoadRe...,A3Q6H2,...,48.0,5.87,No,nrthmntn,http://www.cer-rec.gc.ca/esa-ees/nrthmntn.zip,False,nrthmntn_table-4-summary-of-watercourse-crossi...,1059614_19_lattice-v_1.csv,TABLE 4 SUMMARY OF WATERCOURSE CROSSINGS ALONG...,-1
4,9138,Table,Application for North Montney Project,North Montney,11/8/2013,NOVA Gas Transmission Ltd.,Gas,B2-16 ESA_Appendix_G_Part1of4 (A3Q6H2),https://apps.cer-rec.gc.ca/REGDOCS/Item/LoadRe...,A3Q6H2,...,48.0,5.87,No,nrthmntn,http://www.cer-rec.gc.ca/esa-ees/nrthmntn.zip,False,nrthmntn_table-4-summary-of-watercourse-crossi...,1059614_20_lattice-v_1.csv,TABLE 4 SUMMARY OF WATERCOURSE CROSSINGS ALONG...,-1


In [276]:
# here we will truncate the number of tokens in the table text so that 
# CUDA is not out of memory when we use zero-shot to label our data
# this may or may not be necessary, should revisit later
max_token_num = 128

In [277]:
df_for_labeling['full_text'] = df_for_labeling['text']
for row in df_for_labeling.itertuples(index=True):
    tokenized_table_text = word_tokenize(df_for_labeling['text'][row.Index])
    if len(tokenized_table_text) > max_token_num:
        df_for_labeling['text'][row.Index] = TreebankWordDetokenizer().detokenize(tokenized_table_text[0:max_token_num])
        
df_for_labeling.head(2)

<ipython-input-277-222a4dffab34>:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_for_labeling['text'][row.Index] = TreebankWordDetokenizer().detokenize(tokenized_table_text[0:max_token_num])


,Index,Content Type,Application Name,Application Short Name,Application Filing Date,Company Name,Commodity,File Name,ESA Folder URL,Document Number,...,PDF Size,PDF Outline,Download folder name,Zipped Project Link,Missing CSV,CSV Filename,csvFileName,text,label,full_text
8889,18023,Table,Application for the 2017 NGTL System Expansion,2017 NGTL System Expansion,3/31/2015,NOVA Gas Transmission Ltd.,Gas,B2-15_ESA_Main_Sec_8_Part1of2 - A4K2S0,https://apps.cer-rec.gc.ca/REGDOCS/Item/LoadRe...,A4K2S0,...,6.50,Yes,2017ngtl,http://www.cer-rec.gc.ca/esa-ees/2017ngtl.zip,False,2017ngtl_table-8a-2-reasonably-forseeable-deve...,2758825_359_lattice-v_1.csv,TABLE 8A2 REASONABLY FORSEEABLE DEVELOPMENTS I...,-1,TABLE 8A2 REASONABLY FORSEEABLE DEVELOPMENTS I...
27753,36887,Table,Application for the Northwest Mainline Expansion,Northwest Mainline,4/29/2011,NOVA Gas Transmission Ltd.,Gas,B3-38 - 7212_App8_EPP - A1Y9J1,https://apps.cer-rec.gc.ca/REGDOCS/Item/LoadRe...,A1Y9J1,...,4.43,Yes,nrthwstmnln,http://www.cer-rec.gc.ca/esa-ees/nrthwstmnln.zip,False,nrthwstmnln_table-1-resource-specific-mitigati...,684515_22_lattice-v_1.csv,TABLE 1 RESOURCESPECIFIC MITIGATION TABLE Loca...,-1,TABLE 1 RESOURCESPECIFIC MITIGATION TABLE Loca...


In [278]:
df_train['full_text'] = df_train['text']
for row in df_train.itertuples(index=True):
    tokenized_table_text = word_tokenize(df_train['text'][row.Index])
    if len(tokenized_table_text) > max_token_num:
        df_train['text'][row.Index] = TreebankWordDetokenizer().detokenize(tokenized_table_text[0:max_token_num])
        
df_train.head(2)

<ipython-input-278-e08837a51249>:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_train['text'][row.Index] = TreebankWordDetokenizer().detokenize(tokenized_table_text[0:max_token_num])


,Index,Content Type,Application Name,Application Short Name,Application Filing Date,Company Name,Commodity,File Name,ESA Folder URL,Document Number,...,PDF Size,PDF Outline,Download folder name,Zipped Project Link,Missing CSV,CSV Filename,csvFileName,text,label,full_text
0,9134,Table,Application for North Montney Project,North Montney,11/8/2013,NOVA Gas Transmission Ltd.,Gas,B2-16 ESA_Appendix_G_Part1of4 (A3Q6H2),https://apps.cer-rec.gc.ca/REGDOCS/Item/LoadRe...,A3Q6H2,...,5.87,No,nrthmntn,http://www.cer-rec.gc.ca/esa-ees/nrthmntn.zip,False,nrthmntn_table-3-summary-of-aquatics-field-wor...,1059614_14_lattice-v_1.csv,TABLE 3 SUMMARY OF AQUATICS FIELD WORK AND ABO...,-1,TABLE 3 SUMMARY OF AQUATICS FIELD WORK AND ABO...
1,9135,Table,Application for North Montney Project,North Montney,11/8/2013,NOVA Gas Transmission Ltd.,Gas,B2-16 ESA_Appendix_G_Part1of4 (A3Q6H2),https://apps.cer-rec.gc.ca/REGDOCS/Item/LoadRe...,A3Q6H2,...,5.87,No,nrthmntn,http://www.cer-rec.gc.ca/esa-ees/nrthmntn.zip,False,nrthmntn_table-4-summary-of-watercourse-crossi...,1059614_17_lattice-v_1.csv,TABLE 4 SUMMARY OF WATERCOURSE CROSSINGS ALONG...,-1,TABLE 4 SUMMARY OF WATERCOURSE CROSSINGS ALONG...


In [279]:
for row in df_for_labeling.itertuples():
    tokenized_table_text = word_tokenize(df_for_labeling['text'][row.Index])
    assert len(tokenized_table_text) <= max_token_num, f"You have at least one table with more than {max_token_num} tokens."

In [280]:
df_for_labeling.to_csv(f'./mydata/zero_shot_esa_index_for_labeling_{max_token_num}_max_tokens.csv')

In [281]:
df_joined.to_csv(f'./data/zero_shot_esa_index_train_{max_token_num}_max_tokens.csv')